<a href="https://colab.research.google.com/github/wiroel/my-repo/blob/main/with_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With pipeline

In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/titanic1.csv'

df = pd.read_csv(filepath)

df

Mounted at /content/drive


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df.drop(columns=['Name', 'Cabin', 'PassengerId', 'Ticket'], inplace=True)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [5]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


Imputation transformer
(filling the missing values, fills with mean usually)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2


In [7]:
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]) ,       # [1] is the index value of age, name is not written because at the time of pipelining, the result will be a numpy array and the categorical value may create some problems later on
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder = 'passthrough')                   # remainder=passthrough, otherwise the raminder will be dropped

one hot encoding transformer
there is no drop=first because of decision tree

In [8]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6])
], remainder='passthrough')

minmax scaling transformer

In [9]:
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0, 10))     # slice apply this transformation on all the ten columns that have been created thus far, total=7, instead of age, there is male-female, for embarked there is 3 cities s-c-s, so 7-2+5=10
])

feature selection

In [10]:
trf4 = SelectKBest(score_func=chi2, k=5)      # chi2=scoring function, k=5 is top 5 columns are used for feature scaling

decision tree classifier

In [11]:
trf5 = DecisionTreeClassifier()

Pipelining

In [12]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

Difference between make_pipeline and Pipeline

Pipeline requires user to pass the sp name of transformer and its object
make_pipeline doesnt require such formalities

(same applies to ColumnTransformer and make_column_transformer)

In [13]:
pipe1 = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [14]:
# train

pipe1.fit(X_train, y_train)

pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7e95447c0540>)),
                ('trf5', DecisionTreeClassifier())])

In [15]:
from sklearn import set_config
set_config(display='diagram')

this named_steps might only work with Pipeline and not make_pipeline,,, because 'trf1' is a name which is present in Pipeline as we pass the name of a certain transformer

In [16]:
y_pred = pipe.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.6256983240223464

In [18]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X_train, y_train, cv=6, scoring='accuracy').mean()

np.float64(0.6390471442814414)

GridSearch for hyperparameter tuning

In [19]:
params = {
    'trf5__max_depth':[1,2,3,4,5, None]
}

In [20]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, params, cv=6, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=6,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x7e95447c0540>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [21]:
grid.best_score_

np.float64(0.6390471442814414)

In [22]:
grid.best_params_

{'trf5__max_depth': 1}